# Bone Fracture Detection - Google Colab Setup

This notebook sets up and trains YOLO model for bone fracture detection with GPU support.

## Features:
- ✅ Free GPU access (T4, 16GB VRAM)
- ✅ Complete training pipeline
- ✅ Automatic model download
- ✅ Training visualization


In [ ]:
# Install dependencies
!pip install ultralytics opencv-python opencv-contrib-python scikit-image scipy -q
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q

print("✅ Dependencies installed!")


In [ ]:
# Check GPU
import torch
print("="*60)
print("GPU INFORMATION")
print("="*60)
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram_gb:.2f} GB")
    print("✅ GPU is ready!")
    recommended_model = 's' if vram_gb >= 8 else 'n'
    print(f"Recommended: YOLOv8{recommended_model}")
else:
    print("⚠️  No GPU! Go to: Runtime > Change runtime type > GPU")
print("="*60)


## Step 2: Upload Dataset

**Option A**: Upload from your computer (click folder icon on left)
**Option B**: Mount Google Drive and copy dataset


In [ ]:
# Mount Google Drive (if dataset is in Drive)
from google.colab import drive
drive.mount('/content/drive')

# Or upload dataset.zip manually and unzip:
# !unzip -q /content/archive.zip -d /content/bone_fracture_detection/data/

print("✅ Ready for dataset upload!")


## Step 3: Training Configuration

Modify these settings as needed:


In [ ]:
# Training Configuration
DATA_YAML = "/content/bone_fracture_detection/data/archive/bone fracture detection.v4-v4.yolov8/data.yaml"
MODEL_SIZE = "n"  # Options: 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (xlarge)
EPOCHS = 10       # 5-10 epochs
BATCH = 16        # Adjust based on GPU: 8 for 4GB, 16 for 8GB+, 32 for 16GB+
IMGSZ = 640       # Image size

print(f"Configuration:")
print(f"  Model: YOLOv8{MODEL_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH}")
print(f"  Image Size: {IMGSZ}")


## Step 4: Update Data YAML and Start Training


In [ ]:
from ultralytics import YOLO
import yaml
from pathlib import Path

# Update data.yaml paths
data_yaml_path = Path(DATA_YAML)
if data_yaml_path.exists():
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    base_dir = data_yaml_path.parent
    data_config['train'] = str(base_dir / 'train' / 'images')
    data_config['val'] = str(base_dir / 'valid' / 'images')
    data_config['test'] = str(base_dir / 'test' / 'images')
    
    updated_yaml = base_dir / 'data_updated.yaml'
    with open(updated_yaml, 'w') as f:
        yaml.dump(data_config, f)
    
    DATA_YAML = str(updated_yaml)
    print(f"✅ Data YAML updated: {updated_yaml}")
else:
    print(f"⚠️  Dataset not found. Please upload dataset first!")

# Start Training
print("\n" + "="*80)
print("STARTING TRAINING")
print("="*80)
print(f"This will take ~2-5 hours for {EPOCHS} epochs...")
print("="*80)

model = YOLO(f"yolov8{MODEL_SIZE}.pt")

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project="/content/bone_fracture_detection/yolo_training_results",
    name=f"yolov8{MODEL_SIZE}_bone_fracture",
    save=True,
    plots=True,
    verbose=True
)

print("\n✅ Training Complete!")


In [ ]:
# Display training results
from IPython.display import Image, display

results_dir = f"/content/bone_fracture_detection/yolo_training_results/yolov8{MODEL_SIZE}_bone_fracture"
best_model = f"{results_dir}/weights/best.pt"

try:
    display(Image(f"{results_dir}/results.png"))
    display(Image(f"{results_dir}/confusion_matrix.png"))
    print(f"✅ Best model: {best_model}")
except Exception as e:
    print(f"Results available in: {results_dir}")

# Download model
from google.colab import files
if Path(best_model).exists():
    files.download(best_model)
    print("✅ Model downloaded!")
    
    # Also save to Drive
    !cp {best_model} /content/drive/MyDrive/
    print("✅ Model saved to Google Drive!")
